In [2]:
import json
import numpy as np
import pandas as pd

In [3]:
with open("jin.json", "r", encoding="utf-8") as json_file:
    json_data = json.load(json_file)

In [4]:
data = []
for j in json_data['transaction']:
    sample = {'year':j['year'], 'month':j['month']}
    for a in j['amount']:
        sample[f"{a['type']}_total"] = a['total']
        sample[f"{a['type']}_normal"] = a['normal']
        sample[f"{a['type']}_public_transport"] = a['public_transport']
        sample[f"{a['type']}_cultural_cost"] = a['cultural_cost']
        sample[f"{a['type']}_traditional_market"] = a['traditional_market']
    data.append(sample)
data = pd.DataFrame(data)
data['total'] = data['신용카드_total'] + data['직불카드_total'] + data['현금영수증_total']
data.head()

,year,month,신용카드_total,신용카드_normal,신용카드_public_transport,신용카드_cultural_cost,신용카드_traditional_market,직불카드_total,직불카드_normal,직불카드_public_transport,직불카드_cultural_cost,직불카드_traditional_market,현금영수증_total,현금영수증_normal,현금영수증_public_transport,현금영수증_cultural_cost,현금영수증_traditional_market,total
0,2019,1,68450,0,68450,0,0,326390,318390,0,0,8000,0,0,0,0,0,394840
1,2019,2,55850,0,55850,0,0,343780,292780,27000,24000,0,0,0,0,0,0,399630
2,2019,3,76000,0,76000,0,0,512380,418180,94200,0,0,0,0,0,0,0,588380
3,2019,4,75000,0,75000,0,0,652650,652650,0,0,0,0,0,0,0,0,727650
4,2019,5,77000,0,77000,0,0,432480,432480,0,0,0,0,0,0,0,0,509480


In [5]:
from sklearn.preprocessing import MinMaxScaler
time_step = 12

scaler = MinMaxScaler()
scaler.fit(data[['total']])
x = scaler.transform(data.iloc[-time_step:,-1:].values)
x = x.reshape(1, time_step, 1)
x

array([[[0.41815745],
        [0.30227363],
        [0.16166659],
        [0.07198298],
        [0.0966362 ],
        [0.2012888 ],
        [0.48304371],
        [0.39430039],
        [0.24563988],
        [0.18033495],
        [0.19749336],
        [0.55239803]]])

In [6]:
from tensorflow.keras.models import load_model

model = load_model('./model/model.h5')

In [7]:
y_pred = scaler.inverse_transform(model.predict(x))
y_pred = y_pred.astype('int')

In [10]:
salary = 2500

salary *= 10000

if salary >= 15000000:
    minimum_deductible_amount = salary * 0.25
else:
    minimum_deductible_amount = salary * 0.20

if salary <= 70000000:
    deduction_limit = 3000000
elif salary <= 120000000:
    deduction_limit = 2500000
else:
    deduction_limit = 2000000

remain_month = data.iloc[-1, 1]
year_total_data = data.iloc[-remain_month:,:].sum()

used_credit_card_expense = year_total_data['신용카드_normal']
used_debit_card_expense = year_total_data['직불카드_normal'] + year_total_data['현금영수증_normal']

used_public_transport_expense = year_total_data['신용카드_public_transport'] + year_total_data['직불카드_public_transport'] + year_total_data['현금영수증_public_transport']
if used_public_transport_expense > 2500000:
    if year_total_data['신용카드_public_transport'] - 2500000 > 0:
        used_credit_card_expense += year_total_data['신용카드_public_transport'] - 2500000
        used_debit_card_expense += year_total_data['직불카드_public_transport'] + year_total_data['현금영수증_public_transport']
    else:
        used_debit_card_expense += used_public_transport_expense - 2500000

if salary > 70000000:
    used_credit_card_expense += year_total_data['신용카드_cultural_cost']
    used_debit_card_expense += year_total_data['직불카드_cultural_cost'] + year_total_data['현금영수증_cultural_cost']
else:
    used_cultural_cost_expense = year_total_data['신용카드_cultural_cost'] + year_total_data['직불카드_cultural_cost'] + year_total_data['현금영수증_cultural_cost']
    if used_cultural_cost_expense > 3333333:
        if year_total_data['신용카드_cultural_cost'] - 3333333 > 0:
            used_credit_card_expense += year_total_data['신용카드_cultural_cost'] - 3333333
            used_debit_card_expense += year_total_data['직불카드_cultural_cost'] + year_total_data['현금영수증_cultural_cost']
        else:
            used_debit_card_expense += used_cultural_cost_expense - 3333333

used_traditional_market_expense = year_total_data['신용카드_traditional_market'] + year_total_data['직불카드_traditional_market'] + year_total_data['현금영수증_traditional_market']
if used_traditional_market_expense > 2500000:
    if year_total_data['신용카드_traditional_market'] - 2500000 > 0:
        used_credit_card_expense += year_total_data['신용카드_traditional_market'] - 2500000
        used_debit_card_expense += year_total_data['직불카드_traditional_market'] + year_total_data['현금영수증_traditional_market']
    else:
        used_debit_card_expense += used_traditional_market_expense - 2500000

total_used_expense = used_credit_card_expense + used_debit_card_expense
total_to_use_expense = y_pred[0, :12-remain_month].sum()

total_expense = total_used_expense + total_to_use_expense

if total_expense <= minimum_deductible_amount:
    to_use_credit_card_expense = total_to_use_expense
    to_use_debit_card_expense = 0
    income_deduction = 0
    income_deduction_credit_card = 0
elif total_expense - minimum_deductible_amount <= deduction_limit * 10 // 3:
    if used_credit_card_expense > minimum_deductible_amount:
        to_use_credit_card_expense = 0
        to_use_debit_card_expense = total_to_use_expense
        income_deduction = (used_credit_card_expense + to_use_credit_card_expense - minimum_deductible_amount) * 0.15 + (used_debit_card_expense + to_use_debit_card_expense) * 0.30
    elif used_debit_card_expense > total_expense - minimum_deductible_amount:
        to_use_credit_card_expense = total_to_use_expense
        to_use_debit_card_expense = 0
        if used_credit_card_expense + to_use_credit_card_expense - minimum_deductible_amount > 0:
            income_deduction = (used_credit_card_expense + to_use_credit_card_expense - minimum_deductible_amount) * 0.15 + (used_debit_card_expense + to_use_debit_card_expense) * 0.30
        else:
            income_deduction = (total_expense - minimum_deductible_amount) * 0.30
    else:
        to_use_credit_card_expense = (minimum_deductible_amount - used_credit_card_expense)
        to_use_debit_card_expense = (total_expense - minimum_deductible_amount - used_debit_card_expense)
        income_deduction = (total_expense - minimum_deductible_amount) * 0.30
    income_deduction_credit_card = (total_expense - minimum_deductible_amount) * 0.15
else:
    excess_amount = total_expense - minimum_deductible_amount - deduction_limit * 10 // 3
    to_use_credit_card_expense = (minimum_deductible_amount + 2 * excess_amount - used_credit_card_expense)
    to_use_debit_card_expense = (deduction_limit * 10 // 3 - excess_amount - used_debit_card_expense)
    income_deduction = deduction_limit
    if (total_expense - minimum_deductible_amount) * 0.15 < deduction_limit:
        income_deduction_credit_card = (total_expense - minimum_deductible_amount) * 0.15
    else:
        income_deduction_credit_card = deduction_limit

if salary <= 5000000:
    earned_income_amount = salary - salary * 0.70
elif salary <= 15000000:
    earned_income_amount = salary - (3500000 + (salary - 5000000) * 0.40)
elif salary <= 45000000:
    earned_income_amount = salary - (7500000 + (salary - 15000000) * 0.15)
elif salary <= 100000000:
    earned_income_amount = salary - (12000000 + (salary - 45000000) * 0.05)
else:
    earned_income_amount = salary - (14750000 + (salary - 100000000) * 0.02)

tax_base = earned_income_amount - income_deduction
if tax_base <= 12000000:
    tariff = 0.06
elif tax_base <= 46000000:
    tariff = 0.15
elif tax_base <= 88000000:
    tariff = 0.24
elif tax_base <= 150000000:
    tariff = 0.35
elif tax_base <= 300000000:
    tariff = 0.38
elif tax_base <= 500000000:
    tariff = 0.40
else:
    tariff = 0.42


print("총급여 : ", salary)
print("근로소득금액 : ", earned_income_amount)
print("카드소득공제 최저사용금액 : ", minimum_deductible_amount)
print("적용과세표준 : ", tariff)
print("나의 연간 카드사용 예측금액 : ", total_expense)
print("연말까지 사용할 예측금액 : ", total_to_use_expense)
print("황금비율 신용카드(연말까지 사용할 금액) : ", round(to_use_credit_card_expense))
print("황금비율 체크카드(연말까지 사용할 금액) : ", round(to_use_debit_card_expense))
print("황금비율 신용카드(월) : ", round(to_use_credit_card_expense) // remain_month)
print("황금비율 체크카드(월) : ", round(to_use_debit_card_expense) // remain_month)
print("실제 공제대상 : ", round(income_deduction))
print("실제 세제혜택 : ", round(income_deduction * tariff))
print("카드사용금액을 전부 신용카드로만 썼을 때 세제혜택 : ", round(income_deduction_credit_card * tariff))

총급여 :  25000000
근로소득금액 :  16000000.0
카드소득공제 최저사용금액 :  6250000.0
적용과세표준 :  0.15
나의 연간 카드사용 예측금액 :  6896586
연말까지 사용할 예측금액 :  4276336
황금비율 신용카드(연말까지 사용할 금액) :  3629750
황금비율 체크카드(연말까지 사용할 금액) :  646586
황금비율 신용카드(월) :  725950
황금비율 체크카드(월) :  129317
실제 공제대상 :  193976
실제 세제혜택 :  29096
카드사용금액을 전부 신용카드로만 썼을 때 세제혜택 :  14548
